# Introduction

In this project, you will build a neural network of your own design to evaluate the CIFAR-10 dataset.

To meet the requirements for this project, you will need to achieve an accuracy greater than 45%. 
If you want to beat Detectocorp's algorithm, you'll need to achieve an accuracy greater than 70%. 
(Beating Detectocorp's algorithm is not a requirement for passing this project, but you're encouraged to try!)

Some of the benchmark results on CIFAR-10 include:

78.9% Accuracy | [Deep Belief Networks; Krizhevsky, 2010](https://www.cs.toronto.edu/~kriz/conv-cifar10-aug2010.pdf)

90.6% Accuracy | [Maxout Networks; Goodfellow et al., 2013](https://arxiv.org/pdf/1302.4389.pdf)

96.0% Accuracy | [Wide Residual Networks; Zagoruyko et al., 2016](https://arxiv.org/pdf/1605.07146.pdf)

99.0% Accuracy | [GPipe; Huang et al., 2018](https://arxiv.org/pdf/1811.06965.pdf)

98.5% Accuracy | [Rethinking Recurrent Neural Networks and other Improvements for ImageClassification; Nguyen et al., 2020](https://arxiv.org/pdf/2007.15161.pdf)

Research with this dataset is ongoing. Notably, many of these networks are quite large and quite expensive to train. 

## Imports

In [1]:
## This cell contains the essential imports you will need – DO NOT CHANGE THE CONTENTS! ##
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms

In [2]:
# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 20
# define the train and val splits
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load the Dataset

Specify your transforms as a list first.
The transforms module is already loaded as `transforms`.

CIFAR-10 is fortunately included in the torchvision module.
Then, you can create your dataset using the `CIFAR10` object from `torchvision.datasets` ([the documentation is available here](https://pytorch.org/vision/stable/generated/torchvision.datasets.CIFAR10.html)).
Make sure to specify `download=True`! 

Once your dataset is created, you'll also need to define a `DataLoader` from the `torch.utils.data` module for both the train and the test set.

In [3]:
# Define transforms
## YOUR CODE HERE ##

BATCH_SIZE = 64

transform = transforms.Compose([ transforms.ToTensor()])# The output of torchvision datasets are PILImage images of range [0, 1]

# Create training set and define training dataloader
## YOUR CODE HERE ##

trainset = datasets.CIFAR10('data', train=True, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers = 2)

# Create test set and define test dataloader
## YOUR CODE HERE ##

testset = datasets.CIFAR10(root='data/', download=True, train=False, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True, num_workers = 2)

trainSteps = len(trainloader.dataset) // BATCH_SIZE


# The 10 classes in the dataset
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Explore the Dataset
Using matplotlib, numpy, and torch, explore the dimensions of your data.

You can view images using the `show5` function defined below – it takes a data loader as an argument.
Remember that normalized images will look really weird to you! You may want to try changing your transforms to view images.
Typically using no transforms other than `toTensor()` works well for viewing – but not as well for training your network.
If `show5` doesn't work, go back and check your code for creating your data loaders and your training/test sets.

In [ ]:
def show5(img_loader):
    dataiter = iter(img_loader)
    
    batch = next(dataiter)
    labels = batch[1][0:5]
    images = batch[0][0:5]
    for i in range(5):
        print(classes[labels[i]])
    
        image = images[i].numpy()
        print(image.shape)
        plt.imshow(np.rot90(image.T, k=3))
        plt.show()

In [ ]:
# Explore data
## YOUR CODE HERE ##

show5(train_loader)

## Build your Neural Network
Using the layers in `torch.nn` (which has been imported as `nn`) and the `torch.nn.functional` module (imported as `F`), construct a neural network based on the parameters of the dataset. 
Feel free to construct a model of any architecture – feedforward, convolutional, or even something more advanced!

In [4]:
# TODO: Define your network architecture here
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3072, 1536)
        self.fc2 = nn.Linear(1536, 786)
        self.fc3 = nn.Linear(786, 256)
        self.fc3 = nn.Linear(786, 512)
        self.fc4 = nn.Linear(512, 265)
        self.fc5 = nn.Linear(265, 128)
        self.fc6 = nn.Linear(128, 64)
        self.fc7 = nn.Linear(64, 10)

        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.log_softmax(self.fc7(x), dim=1)
        
        return x

Specify a loss function and an optimizer, and instantiate the model.

If you use a less common loss function, please note why you chose that loss function in a comment.

In [5]:
## YOUR CODE HERE ##

# initialize the LeNet model
print("[INFO] initializing the LeNet model...")
model = Classifier()

# initialize our optimizer and loss function
# optimizer = optim.Adam(model.parameters(), lr=0.003)
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9, weight_decay = 1e-4)
criterion = nn.NLLLoss()

# initialize a dictionary to store training history
H = {
    "train_losses": [],
    "train_acc": []
}



[INFO] initializing the LeNet model...


## Running your Neural Network
Use whatever method you like to train your neural network, and ensure you record the average loss at each epoch. 
Don't forget to use `torch.device()` and the `.to()` method for both your model and your data if you are using GPU!

If you want to print your loss during each epoch, you can use the `enumerate` function and print the loss after a set number of batches. 250 batches works well for most people!

In [6]:
## YOUR CODE HERE ##
# loop over our epochs

for e in range(0, EPOCHS):
    # set the model in training mode
    model.train()
    # initialize the total training and validation loss
    totalTrainLoss = 0
    # initialize the number of correct predictions in the training
    # and validation step
    trainCorrect = 0
    # loop over the training set
    for (x, y) in trainloader:
        # send the input to the device
        (x, y) = (x.to(device), y.to(device))
        # perform a forward pass and calculate the training loss
        pred = model(x)
        loss = criterion(pred, y)
        # zero out the gradients, perform the backpropagation step,
        # and update the weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # add the loss to the total training loss so far and
        # calculate the number of correct predictions
        totalTrainLoss += loss
        trainCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()

    # calculate the average training and validation loss
    avgTrainLoss = totalTrainLoss / trainSteps
    # calculate the training and validation accuracy
    trainCorrect = trainCorrect / len(trainloader.dataset)
    # update our training history
    H["train_losses"].append(avgTrainLoss.cpu().detach().numpy())
    H["train_acc"].append(trainCorrect)
    # print the model training and validation information
    print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(avgTrainLoss, trainCorrect))

[INFO] EPOCH: 1/20
Train loss: 2.238302, Train accuracy: 0.1347
[INFO] EPOCH: 2/20
Train loss: 1.970393, Train accuracy: 0.2406
[INFO] EPOCH: 3/20
Train loss: 1.815759, Train accuracy: 0.3244
[INFO] EPOCH: 4/20
Train loss: 1.705420, Train accuracy: 0.3739
[INFO] EPOCH: 5/20
Train loss: 1.638626, Train accuracy: 0.4027
[INFO] EPOCH: 6/20
Train loss: 1.581100, Train accuracy: 0.4277
[INFO] EPOCH: 7/20
Train loss: 1.537764, Train accuracy: 0.4437
[INFO] EPOCH: 8/20
Train loss: 1.501991, Train accuracy: 0.4585
[INFO] EPOCH: 9/20
Train loss: 1.458921, Train accuracy: 0.4765
[INFO] EPOCH: 10/20
Train loss: 1.417207, Train accuracy: 0.4914
[INFO] EPOCH: 11/20
Train loss: 1.378106, Train accuracy: 0.5037
[INFO] EPOCH: 12/20
Train loss: 1.348762, Train accuracy: 0.5160
[INFO] EPOCH: 13/20
Train loss: 1.324275, Train accuracy: 0.5262
[INFO] EPOCH: 14/20
Train loss: 1.284790, Train accuracy: 0.5403
[INFO] EPOCH: 15/20
Train loss: 1.253390, Train accuracy: 0.5518
[INFO] EPOCH: 16/20
Train loss: 1.

Plot the training loss (and validation loss/accuracy, if recorded).

In [ ]:
## YOUR CODE HERE ##

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.plot(H["train_losses"], label='Training loss')
plt.plot(H['train_acc'], label='Training accuracy')
plt.legend(frameon=False)

## Testing your model
Using the previously created `DataLoader` for the test set, compute the percentage of correct predictions using the highest probability prediction. 

If your accuracy is over 70%, great work! 
This is a hard task to exceed 70% on.

If your accuracy is under 45%, you'll need to make improvements.
Go back and check your model architecture, loss function, and optimizer to make sure they're appropriate for an image classification task.

In [7]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9)

epochs = 20

train_losses, test_losses = [], []
for e in range(epochs):
    tot_train_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        tot_train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    else:
        tot_test_loss = 0
        test_correct = 0  # Number of correct predictions on the test set
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            for images, labels in testloader:
                log_ps = model(images)
                loss = criterion(log_ps, labels)
                tot_test_loss += loss.item()

                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                test_correct += equals.sum().item()

        # Get mean loss to enable comparison between train and test sets
        train_loss = tot_train_loss / len(trainloader.dataset)
        test_loss = tot_test_loss / len(testloader.dataset)

        # At completion of epoch
        train_losses.append(train_loss)
        test_losses.append(test_loss)

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_loss),
              "Test Loss: {:.3f}.. ".format(test_loss),
              "Test Accuracy: {:.3f}".format(test_correct / len(testloader.dataset)))

Epoch: 1/20..  Training Loss: 0.035..  Test Loss: 0.032..  Test Accuracy: 0.202
Epoch: 2/20..  Training Loss: 0.030..  Test Loss: 0.031..  Test Accuracy: 0.285
Epoch: 3/20..  Training Loss: 0.028..  Test Loss: 0.027..  Test Accuracy: 0.364
Epoch: 4/20..  Training Loss: 0.026..  Test Loss: 0.026..  Test Accuracy: 0.382
Epoch: 5/20..  Training Loss: 0.025..  Test Loss: 0.025..  Test Accuracy: 0.420
Epoch: 6/20..  Training Loss: 0.024..  Test Loss: 0.025..  Test Accuracy: 0.438
Epoch: 7/20..  Training Loss: 0.024..  Test Loss: 0.024..  Test Accuracy: 0.465
Epoch: 8/20..  Training Loss: 0.023..  Test Loss: 0.025..  Test Accuracy: 0.430
Epoch: 9/20..  Training Loss: 0.023..  Test Loss: 0.024..  Test Accuracy: 0.459
Epoch: 10/20..  Training Loss: 0.022..  Test Loss: 0.023..  Test Accuracy: 0.483
Epoch: 11/20..  Training Loss: 0.021..  Test Loss: 0.022..  Test Accuracy: 0.501
Epoch: 12/20..  Training Loss: 0.021..  Test Loss: 0.022..  Test Accuracy: 0.486
Epoch: 13/20..  Training Loss: 0.020.

## Saving your model
Using `torch.save`, save your model for future loading.

In [8]:
## YOUR CODE HERE ##
print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

Our model: 

 Classifier(
  (fc1): Linear(in_features=3072, out_features=1536, bias=True)
  (fc2): Linear(in_features=1536, out_features=786, bias=True)
  (fc3): Linear(in_features=786, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=265, bias=True)
  (fc5): Linear(in_features=265, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=10, bias=True)
) 

The state dict keys: 

 odict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias', 'fc4.weight', 'fc4.bias', 'fc5.weight', 'fc5.bias', 'fc6.weight', 'fc6.bias', 'fc7.weight', 'fc7.bias'])


In [9]:
PATH = 'cifar_net.pth'
torch.save(model.state_dict(), PATH)

## Make a Recommendation

Based on your evaluation, what is your recommendation on whether to build or buy? Explain your reasoning below.

Some things to consider as you formulate your recommendation:
* How does your model compare to Detectocorp's model?
* How does it compare to the far more advanced solutions in the literature? 
* What did you do to get the accuracy you achieved? 
* Is it necessary to improve this accuracy? If so, what sort of work would be involved in improving it?



<p> Compared to Detectcorp's model, my model is slightly better, but in real world, it is just a very weak classsifier that need to be tuned more and more and the usage of the convolution neural network will make a great difference in this classifier.

<p> Compared to other state of the art models, it is very bad model, even it can not be used as a reference.

<p> I tried to make a deep network, but it is not effecient to extract the features from the images and make prediciton. I tried to use deeper ntwrok and try to use the momentum in Gradient descent to escape away from the saddle point(local minima) and reach the Global minima.

<p> Tha main problem here is we can not extract the features that used in training of the model. Conv Neural netwrok was inveted to extract the most important features in the images tahat could help in prediction



## Submit Your Project

When you are finished editing the notebook and are ready to turn it in, simply click the **SUBMIT PROJECT** button in the lower right.

Once you submit your project, we'll review your work and give you feedback if there's anything that you need to work on. If you'd like to see the exact points that your reviewer will check for when looking at your work, you can have a look over the project [rubric](https://review.udacity.com/#!/rubrics/3077/view).